# Example code for making plots from test data

In [199]:
import numpy as np
import pandas as pd
import plotly.express as px


In [200]:
dataFrame = pd.read_excel("testData.xlsx",sheet_name="Sheet1")
dataFrame.head()

# drop unnecessary rows in data
# dataFrame.drop(np.arange(0,100), inplace=True)
# dataFrame.drop(np.arange(5000,6000), inplace=True)

,unit_number,time_in_cycles,operational_setting_1,operational_setting_2,operational_setting_3,(Fan inlet temperature) (◦R),(LPC outlet temperature) (◦R),(HPC outlet temperature) (◦R),(LPT outlet temperature) (◦R),(Fan inlet Pressure) (psia),...,(Corrected fan speed) (rpm),(Corrected core speed) (rpm),(Bypass Ratio),(Burner fuel-air ratio),(Bleed Enthalpy),(Required fan speed),(Required fan conversion speed),(High-pressure turbines Cool air flow),(Low-pressure turbines Cool air flow),RUL
0,1,1,-0.0007,-0.0004,100,518.67,641.82,1589.70,1400.60,14.62,...,2388.02,8138.62,8.4195,0.03,392,2388,100,39.06,23.4190,191
1,1,2,0.0019,-0.0003,100,518.67,642.15,1591.82,1403.14,14.62,...,2388.07,8131.49,8.4318,0.03,392,2388,100,39.00,23.4236,190
2,1,3,-0.0043,0.0003,100,518.67,642.35,1587.99,1404.20,14.62,...,2388.03,8133.23,8.4178,0.03,390,2388,100,38.95,23.3442,189
3,1,4,0.0007,0.0000,100,518.67,642.35,1582.79,1401.87,14.62,...,2388.08,8133.83,8.3682,0.03,392,2388,100,38.88,23.3739,188
4,1,5,-0.0019,-0.0002,100,518.67,642.37,1582.85,1406.22,14.62,...,2388.04,8133.80,8.4294,0.03,393,2388,100,38.90,23.4044,187


## Plotly Line Plots

plot multiple columns of data against a target column (e.g. time)

In [201]:
def plot_multiple_variables_vs_target(df, target_column, y_columns):
   """
   Generate an interactive plot of multiple variables against the target column using Plotly.

   Parameters:
       df (pd.DataFrame): The DataFrame containing the data.
       target_column (str): The column to use for the x-axis (e.g., time).
       y_columns (list): The list of columns to plot against the target column.
   """
   # Ensure the target column exists in the DataFrame
   if target_column not in df.columns:
       raise ValueError(f"Target column '{target_column}' not found in DataFrame.")

   # Ensure all y_columns exist in the DataFrame
   for col in y_columns:
       if col not in df.columns:
           raise ValueError(f"Column '{col}' not found in DataFrame.")
    
    #convert to numeric 
   df[target_column] = pd.to_numeric(df[target_column], errors='coerce')

   for col in y_columns:
       df[col] = pd.to_numeric(df[col], errors="coerce")
   
   # Melt the DataFrame to create a long format for Plotly
   melted_df = df.melt(id_vars=target_column, value_vars=y_columns, 
                       var_name="Variable", value_name="Value")

   # Create a plot using Plotly
   fig = px.line(
       melted_df,
       x=target_column,
       y="Value",
       color="Variable",
       title=f"{', '.join(y_columns)} vs {target_column}",
       labels={target_column: target_column, "Value": "Value", "Variable": "Variable"}
   )

   # Show the interactive plot
   fig.show()




In [202]:
# Call the function to plot multiple variables vs target
plot_multiple_variables_vs_target(df=dataFrame, target_column="time_in_cycles", y_columns=["(LPC outlet temperature) (◦R)", "(LPT outlet temperature) (◦R)", "(HPC outlet temperature) (◦R)"])


In [203]:
plot_multiple_variables_vs_target(df=dataFrame, target_column="(HPC outlet pressure) (psia)", y_columns=["(Fan inlet temperature) (◦R)", "(LPC outlet temperature) (◦R)"])

## Plotly 3D scatter

In [204]:
import plotly.graph_objects as go

def plot_3d_scatter_plotly(df, x_col, y_col, z_col, param, width=900, height=700, title=None, colorscale = "viridis", template = "plotly_dark"):
    """
    Creates an interactive 3D scatter plot using Plotly with improved scaling and sizing.

    Parameters:
        df (pd.DataFrame): DataFrame containing the data.
        x_col (str): Column name for the x-axis (e.g., "Mach").
        y_col (str): Column name for the y-axis (e.g., "Altitude").
        z_col (str): Column name for the z-axis (e.g., "Tt" or Temperature Offset).
        param (str): Column name of the parameter to visualize (e.g., "Power_Output"). 
        width (int): Width of the plot (default is 900 pixels).
        height (int): Height of the plot (default is 700 pixels).
        colorscale (str): color of the data points. ("viridis", "sunset" etc.)
        template (str): background color. ("plotly", "plotly_white", "plotly_dark", "ggplot2", "seaborn", "simple_white", "none")
    """
    # Ensure the parameter column is numeric and remove any NaNs
    df[param] = pd.to_numeric(df[param], errors="coerce")
    df = df.dropna(subset=[param])

    # Calculate axis ranges to include all data
    x_range = [df[x_col].min(), df[x_col].max()]
    y_range = [df[y_col].min(), df[y_col].max()]
    z_range = [df[z_col].min(), df[z_col].max()]

    # Create the 3D scatter plot
    fig = go.Figure(data=[go.Scatter3d(
        x = df[x_col],
        y = df[y_col],
        z = df[z_col],
        mode = 'markers',
        marker = dict(
            size = 6,
            color = df[param],        # Color by the parameter values
            colorscale = colorscale,     # Choose a colorscale from Plotly's list
            colorbar = dict(title=param),
            opacity = 0.8,
        )
    )])

    # Update layout for proper sizing and axis scaling
    fig.update_layout(
        title=title,
        width=width,
        height=height,
        template=template,
        scene=dict(
            xaxis=dict(title=x_col, range=x_range, autorange=False),
            yaxis=dict(title=y_col, range=y_range, autorange=False),
            zaxis=dict(title=z_col, range=z_range, autorange=False)
        ),
        margin=dict(l=50, r=50, b=50, t=50)
    )

    fig.show()

In [205]:
plot_3d_scatter_plotly(dataFrame , x_col = "time_in_cycles", y_col = "(Physical fan speed) (rpm)", z_col = "(LPT outlet temperature) (◦R)", param="(Physical core speed) (rpm)", width=900, height=700, title="Example")